# Lake and Lean Installation Guide

This section covers installing the complete Lean 4 toolchain including Lake (the build system).

## Installation Steps for macOS (ARM64)

Lake comes bundled with Lean 4, so we need to install the complete toolchain using `elan` (the Lean version manager).

## ✅ Installation Complete!

**Lake and Lean 4 have been successfully installed!**

### What was installed:
- **elan**: Lean version manager (v4.1.2)
- **Lean 4**: Lean compiler (v4.23.0-rc2)
- **Lake**: Lean build system (v5.0.0)

### Installation location:
- Installation directory: `~/.elan/`
- Binaries: `~/.elan/bin/`
- Added to PATH in `~/.zshrc`

### Available commands:
- `lake build` - Build the project
- `lake update` - Update dependencies
- `lake clean` - Clean build artifacts
- `lake check-build` - Check build status
- `lean --version` - Check Lean version
- `elan show` - Show installed toolchains

### Next steps:
1. Dependencies are being downloaded (Mathlib, etc.)
2. Once complete, you can use `lake build` to build your AQFT2 project
3. Lake can now generate proper dependency graphs and build information

## 🎯 Lake Commands for Dependency Analysis

Now that Lake is installed and working, you can use these commands for dependency analysis:

### Basic Lake Commands:
- `lake build` - Build the entire project
- `lake check-build` - Check if build targets are configured
- `lake clean` - Clean all build artifacts
- `lake update` - Update all dependencies

### Dependency-specific Commands:
- `lake deps` - Show dependency information
- `lake query --all` - Query all build targets
- `lake show-graph` - Show dependency graph (if available)

### Project Status Commands:
- `lake env lean --version` - Check Lean version in project context
- `lake scripts` - List available scripts
- `lake exe --help` - Help for executable targets

The toolchain has been synchronized with Mathlib (v4.23.0-rc2) so everything should work smoothly now!

# AQFT2 Project Blueprint and Dependency Analysis

This notebook provides tools and methods for creating dependency graphs and project blueprints for the AQFT2 Lean project.

## Available Methods for Dependency Analysis

### 1. **Leanblueprint** (Recommended)
- Official Lean community tool for creating mathematical blueprints
- Generates beautiful web-based documentation with dependency graphs
- Requires `leanblueprint` Python package

### 2. **Lake dependency analysis** 
- Built-in Lean/Lake tooling for dependency analysis
- Requires Lake build system to be properly set up

### 3. **Custom Python analysis**
- Parse import statements directly from .lean files
- Create custom dependency graphs using networkx and matplotlib
- Most flexible but requires manual implementation

### 4. **Graph visualization tools**
- Graphviz for static dependency graphs
- Interactive tools like Gephi for large graphs

Let's start by implementing a custom Python-based dependency analyzer since we don't have Lake available.

In [ ]:
import os
import re
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
import json
from collections import defaultdict

# Set up the project path
PROJECT_ROOT = Path("/Users/mdouglas/Documents/GitHub/aqft2")
AQFT2_DIR = PROJECT_ROOT / "Aqft2"

print(f"Project root: {PROJECT_ROOT}")
print(f"Aqft2 directory: {AQFT2_DIR}")
print(f"Directory exists: {AQFT2_DIR.exists()}")

# List all .lean files in the project
lean_files = list(PROJECT_ROOT.glob("**/*.lean"))
print(f"\nFound {len(lean_files)} .lean files:")
for f in sorted(lean_files)[:10]:  # Show first 10
    print(f"  {f.relative_to(PROJECT_ROOT)}")
if len(lean_files) > 10:
    print(f"  ... and {len(lean_files) - 10} more files")

In [ ]:
class LeanDependencyAnalyzer:
    """Analyzes dependencies in Lean project files."""
    
    def __init__(self, project_root):
        self.project_root = Path(project_root)
        self.dependencies = defaultdict(set)
        self.files = {}
        self.mathlib_imports = defaultdict(set)
        
    def parse_imports(self, file_path):
        """Parse import statements from a Lean file."""
        imports = {"local": set(), "mathlib": set()}
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                
            # Find all import statements
            import_pattern = r'^import\s+(.+)$'
            for match in re.finditer(import_pattern, content, re.MULTILINE):
                import_line = match.group(1).strip()
                
                # Clean up the import
                import_name = import_line.split()[0] if import_line else ""
                
                if import_name.startswith('Mathlib'):
                    imports["mathlib"].add(import_name)
                elif import_name.startswith('«Aqft2»') or import_name.startswith('Aqft2'):
                    # Local project import
                    clean_name = import_name.replace('«', '').replace('»', '')
                    imports["local"].add(clean_name)
                    
        except Exception as e:
            print(f"Error parsing {file_path}: {e}")
            
        return imports
    
    def analyze_project(self):
        """Analyze all .lean files in the project."""
        lean_files = list(self.project_root.glob("**/*.lean"))
        
        for file_path in lean_files:
            rel_path = file_path.relative_to(self.project_root)
            file_key = str(rel_path).replace('.lean', '').replace('/', '.')
            
            imports = self.parse_imports(file_path)
            
            self.files[file_key] = {
                'path': rel_path,
                'local_imports': imports['local'],
                'mathlib_imports': imports['mathlib']
            }
            
            # Build dependency graph
            for local_import in imports['local']:
                self.dependencies[file_key].add(local_import)
                
            # Track mathlib dependencies
            self.mathlib_imports[file_key] = imports['mathlib']
    
    def create_dependency_graph(self):
        """Create NetworkX graph of local dependencies."""
        G = nx.DiGraph()
        
        # Add all files as nodes
        for file_key in self.files:
            G.add_node(file_key)
            
        # Add dependency edges
        for file_key, deps in self.dependencies.items():
            for dep in deps:
                if dep in self.files:  # Only add edges to files that exist
                    G.add_edge(dep, file_key)  # dep -> file_key (dep is imported by file_key)
                    
        return G
    
    def print_summary(self):
        """Print analysis summary."""
        print(f"Project Analysis Summary:")
        print(f"========================")
        print(f"Total .lean files: {len(self.files)}")
        print(f"Files with local dependencies: {len([f for f in self.files if self.dependencies[f]])}")
        
        # Most imported files
        import_counts = defaultdict(int)
        for deps in self.dependencies.values():
            for dep in deps:
                import_counts[dep] += 1
                
        print(f"\nMost imported local files:")
        for file, count in sorted(import_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
            print(f"  {file}: {count} imports")
            
        # Files with most dependencies
        print(f"\nFiles with most local dependencies:")
        for file, deps in sorted([(f, len(self.dependencies[f])) for f in self.files], 
                                key=lambda x: x[1], reverse=True)[:5]:
            print(f"  {file}: {deps} dependencies")

# Create analyzer instance
analyzer = LeanDependencyAnalyzer(PROJECT_ROOT)
print("Analyzer created successfully!")

In [ ]:
# Run the analysis
print("Starting project analysis...")
analyzer.analyze_project()
analyzer.print_summary()

In [ ]:
# Create and visualize the dependency graph
G = analyzer.create_dependency_graph()

print(f"\nDependency Graph Statistics:")
print(f"Nodes (files): {G.number_of_nodes()}")
print(f"Edges (dependencies): {G.number_of_edges()}")
print(f"Is DAG (no circular dependencies): {nx.is_directed_acyclic_graph(G)}")

# Find strongly connected components (circular dependencies)
if not nx.is_directed_acyclic_graph(G):
    sccs = list(nx.strongly_connected_components(G))
    print(f"Strongly connected components: {len(sccs)}")
    for i, scc in enumerate(sccs):
        if len(scc) > 1:
            print(f"  Component {i}: {scc}")

# Create visualization
plt.figure(figsize=(15, 10))

# Use hierarchical layout if it's a DAG
if nx.is_directed_acyclic_graph(G):
    # Topological sort for layered layout
    try:
        pos = nx.nx_agraph.graphviz_layout(G, prog='dot')
    except:
        # Fallback to spring layout
        pos = nx.spring_layout(G, k=3, iterations=50)
else:
    pos = nx.spring_layout(G, k=3, iterations=50)

# Draw the graph
nx.draw(G, pos, 
        with_labels=True, 
        node_color='lightblue',
        node_size=1000,
        font_size=8,
        font_weight='bold',
        arrows=True,
        arrowsize=20,
        edge_color='gray',
        alpha=0.7)

plt.title("AQFT2 Local Dependency Graph", size=16)
plt.axis('off')
plt.tight_layout()
plt.show()

# Print topological order if it's a DAG
if nx.is_directed_acyclic_graph(G):
    print(f"\nTopological order (build order):")
    topo_order = list(nx.topological_sort(G))
    for i, file in enumerate(topo_order):
        print(f"  {i+1:2d}. {file}")

In [ ]:
# Detailed file dependency analysis
print("Detailed Dependency Analysis")
print("="*50)

for file_key in sorted(analyzer.files.keys()):
    file_info = analyzer.files[file_key]
    local_deps = analyzer.dependencies[file_key]
    mathlib_deps = analyzer.mathlib_imports[file_key]
    
    print(f"\n📁 {file_key}")
    print(f"   Path: {file_info['path']}")
    
    if local_deps:
        print(f"   Local dependencies ({len(local_deps)}):")
        for dep in sorted(local_deps):
            print(f"     → {dep}")
    else:
        print(f"   Local dependencies: None")
        
    if mathlib_deps:
        print(f"   Mathlib imports ({len(mathlib_deps)}):")
        # Show first few mathlib imports to avoid clutter
        for dep in sorted(list(mathlib_deps)[:3]):
            print(f"     → {dep}")
        if len(mathlib_deps) > 3:
            print(f"     ... and {len(mathlib_deps) - 3} more")
    else:
        print(f"   Mathlib imports: None")

In [ ]:
# Export dependency information to various formats

# 1. Export to DOT format for Graphviz
def export_to_dot(graph, filename):
    """Export graph to DOT format for Graphviz."""
    nx.drawing.nx_pydot.write_dot(graph, filename)
    print(f"Exported to {filename}")

# 2. Export to JSON for web visualization
def export_to_json(analyzer, filename):
    """Export dependency data to JSON."""
    data = {
        'files': {},
        'dependencies': {},
        'mathlib_imports': {}
    }
    
    for file_key, file_info in analyzer.files.items():
        data['files'][file_key] = {
            'path': str(file_info['path']),
            'local_imports': list(file_info['local_imports']),
            'mathlib_imports': list(file_info['mathlib_imports'])
        }
    
    for file_key, deps in analyzer.dependencies.items():
        data['dependencies'][file_key] = list(deps)
        
    for file_key, mathlib_deps in analyzer.mathlib_imports.items():
        data['mathlib_imports'][file_key] = list(mathlib_deps)
    
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"Exported to {filename}")

# 3. Export build order
def export_build_order(graph, filename):
    """Export topological build order."""
    if nx.is_directed_acyclic_graph(graph):
        topo_order = list(nx.topological_sort(graph))
        with open(filename, 'w') as f:
            f.write("# AQFT2 Build Order (Topological Sort)\n")
            f.write("# Files should be built in this order to respect dependencies\n\n")
            for i, file in enumerate(topo_order):
                f.write(f"{i+1:2d}. {file}\n")
        print(f"Build order exported to {filename}")
    else:
        print("Cannot create build order - graph contains cycles!")

# Create exports
try:
    export_to_dot(G, PROJECT_ROOT / "aqft2_dependencies.dot")
    export_to_json(analyzer, PROJECT_ROOT / "aqft2_dependencies.json")
    export_build_order(G, PROJECT_ROOT / "aqft2_build_order.txt")
    print("\n✅ All exports completed successfully!")
except Exception as e:
    print(f"❌ Export error: {e}")

## Setting up Leanblueprint (Recommended)

For the most professional blueprint creation, install and set up `leanblueprint`:

### Installation Steps:

1. **Install leanblueprint**:
   ```bash
   pip install leanblueprint
   ```

2. **Create blueprint configuration**:
   Create a `blueprint/` directory in your project root with:
   - `blueprint.yaml` - Main configuration
   - `src/` - Blueprint source files
   - `web/` - Generated web files

3. **Initialize blueprint**:
   ```bash
   leanblueprint init
   ```

4. **Generate blueprint**:
   ```bash
   leanblueprint build
   ```

### Blueprint Configuration Example:

The notebook below shows how to create the necessary configuration files.

In [ ]:
# Create leanblueprint configuration files

# Create blueprint directory structure
blueprint_dir = PROJECT_ROOT / "blueprint"
blueprint_src = blueprint_dir / "src"
blueprint_web = blueprint_dir / "web"

blueprint_dir.mkdir(exist_ok=True)
blueprint_src.mkdir(exist_ok=True)
blueprint_web.mkdir(exist_ok=True)

# Create blueprint.yaml configuration
blueprint_config = """title: "Algebraic Quantum Field Theory (AQFT2)"
author: "MRD and SH"
description: "Formalization of Algebraic Quantum Field Theory using the Glimm-Jaffe approach"

# Project settings
lean_project: "."
github_repo: "mrdouglasny/aqft2"

# Build settings
build_dir: "blueprint/web"
src_dir: "blueprint/src"

# Chapters and sections
chapters:
  - title: "Foundations"
    sections:
      - title: "Basic Framework"
        file: "basic.md"
        lean_file: "Aqft2/Basic.lean"
      - title: "Functional Analysis"
        file: "functional_analysis.md"
        lean_file: "Aqft2/FunctionalAnalysis.lean"
      - title: "Hilbert Spaces"
        file: "hilbert_space.md"
        lean_file: "Aqft2/HilbertSpace.lean"
        
  - title: "Quantum Field Theory"
    sections:
      - title: "QFT Hilbert Space"
        file: "qft_hilbert.md"
        lean_file: "Aqft2/QFTHilbertSpace.lean"
      - title: "Operators"
        file: "operators.md"
        lean_file: "Aqft2/Operators.lean"
      - title: "Euclidean Theory"
        file: "euclidean.md"
        lean_file: "Aqft2/Euclidean.lean"
        
  - title: "Advanced Topics"
    sections:
      - title: "Osterwalder-Schrader Axioms"
        file: "os_axioms.md"
        lean_file: "Aqft2/OS_Axioms.lean"
      - title: "OS4 Implementation"
        file: "os4.md" 
        lean_file: "Aqft2/OS4.lean"
      - title: "Gaussian Free Field"
        file: "gaussian_free_field.md"
        lean_file: "Aqft2/GaussianFreeField.lean"

# Dependency graph settings
dependency_graph:
  enabled: true
  format: "svg"
  include_mathlib: false
"""

with open(blueprint_dir / "blueprint.yaml", "w") as f:
    f.write(blueprint_config)

print("✅ Created blueprint.yaml configuration")

# Create sample blueprint source files
basic_md = """# Basic Framework

This chapter establishes the foundational definitions for the Glimm-Jaffe approach to Algebraic Quantum Field Theory.

## Key Components

### Spacetime Structure
- Spacetime dimension: 4D Euclidean space
- Test function spaces using Schwartz functions
- Measure theory foundations

### Field Configurations
- Tempered distributions as field configurations
- Generating functionals
- Support properties

## Main Definitions

```lean
-- Reference to Lean definitions
def SpaceTime := EuclideanSpace ℝ (Fin 4)
def TestFunctionSpace := 𝒮(ℝ^4, ℝ)
```

This framework provides the mathematical foundation for constructing quantum field theories rigorously.
"""

with open(blueprint_src / "basic.md", "w") as f:
    f.write(basic_md)

print("✅ Created sample blueprint source file")
print(f"📁 Blueprint directory created at: {blueprint_dir}")
print(f"\nNext steps:")
print(f"1. Install leanblueprint: pip install leanblueprint")
print(f"2. Navigate to project root: cd {PROJECT_ROOT}")
print(f"3. Build blueprint: leanblueprint build")
print(f"4. Serve locally: leanblueprint serve")

## Alternative: Using Graphviz for Static Dependency Graphs

If you have Graphviz installed, you can create professional static dependency graphs:

In [ ]:
# Create a more detailed DOT file for Graphviz visualization
def create_detailed_dot_file(analyzer, graph, filename):
    """Create a detailed DOT file with styling and grouping."""
    
    dot_content = """digraph AQFT2_Dependencies {
    // Graph settings
    rankdir=TB;
    node [shape=box, style=filled, fontname="Arial"];
    edge [fontname="Arial"];
    
    // Graph title
    labelloc="t";
    label="AQFT2 Project Dependency Graph";
    fontsize=16;
    fontname="Arial Bold";
    
    // Subgraph for foundational modules
    subgraph cluster_foundation {
        label="Foundation";
        style=filled;
        color=lightgrey;
        
"""
    
    # Categorize files
    foundation_files = ['Aqft2.Basic', 'Aqft2.FunctionalAnalysis', 'Aqft2.HilbertSpace', 'Aqft2.TopologyLemmas']
    qft_files = ['Aqft2.QFTHilbertSpace', 'Aqft2.Operators', 'Aqft2.Euclidean']
    advanced_files = ['Aqft2.OS_Axioms', 'Aqft2.OS4', 'Aqft2.GaussianFreeField', 'Aqft2.HadamardExp']
    
    # Add foundation nodes
    for file in foundation_files:
        if file in analyzer.files:
            dot_content += f'        "{file}" [fillcolor=lightblue];\n'
    
    dot_content += """    }
    
    // Subgraph for QFT modules
    subgraph cluster_qft {
        label="Quantum Field Theory";
        style=filled;
        color=lightgreen;
        
"""
    
    # Add QFT nodes
    for file in qft_files:
        if file in analyzer.files:
            dot_content += f'        "{file}" [fillcolor=lightgreen];\n'
    
    dot_content += """    }
    
    // Subgraph for advanced modules
    subgraph cluster_advanced {
        label="Advanced Topics";
        style=filled;
        color=lightyellow;
        
"""
    
    # Add advanced nodes
    for file in advanced_files:
        if file in analyzer.files:
            dot_content += f'        "{file}" [fillcolor=lightyellow];\n'
    
    dot_content += """    }
    
    // Other files
"""
    
    # Add remaining files
    all_categorized = set(foundation_files + qft_files + advanced_files)
    for file in analyzer.files:
        if file not in all_categorized:
            dot_content += f'    "{file}" [fillcolor=white];\n'
    
    # Add edges
    dot_content += "\n    // Dependencies\n"
    for file, deps in analyzer.dependencies.items():
        for dep in deps:
            if dep in analyzer.files:
                dot_content += f'    "{dep}" -> "{file}";\n'
    
    dot_content += "}\n"
    
    with open(filename, 'w') as f:
        f.write(dot_content)
    
    return filename

# Create the detailed DOT file
dot_file = create_detailed_dot_file(analyzer, G, PROJECT_ROOT / "aqft2_detailed.dot")
print(f"✅ Created detailed DOT file: {dot_file}")

print(f"""
🎯 To generate images from the DOT file:

1. Install Graphviz:
   - macOS: brew install graphviz
   - Ubuntu: sudo apt-get install graphviz
   - Windows: Download from https://graphviz.org/download/

2. Generate images:
   dot -Tpng aqft2_detailed.dot -o aqft2_dependencies.png
   dot -Tsvg aqft2_detailed.dot -o aqft2_dependencies.svg
   dot -Tpdf aqft2_detailed.dot -o aqft2_dependencies.pdf

3. For interactive viewing:
   xdot aqft2_detailed.dot  # Linux
   # Or use online viewers like dreampuf.github.io/GraphvizOnline/
""")

## Summary: Blueprint and Dependency Graph Options

This notebook provides multiple approaches for creating blueprints and dependency graphs for your AQFT2 Lean project:

### 1. **Leanblueprint** (🏆 Recommended)
- **Pros**: Professional, web-based, integrated with Lean ecosystem
- **Setup**: `pip install leanblueprint`, configure `blueprint.yaml`
- **Output**: Interactive web documentation with dependency graphs
- **Best for**: Official project documentation and publication

### 2. **Custom Python Analysis** (✅ Implemented above)
- **Pros**: Flexible, customizable, works without external tools
- **Features**: Dependency parsing, graph analysis, multiple export formats
- **Output**: NetworkX graphs, JSON data, build orders
- **Best for**: Development and debugging dependency issues

### 3. **Graphviz Visualization** (🎨 Professional graphs)
- **Pros**: High-quality static graphs, publication-ready
- **Setup**: Install Graphviz, use generated DOT files
- **Output**: PNG, SVG, PDF dependency graphs
- **Best for**: Papers, presentations, documentation

### 4. **Lake Integration** (⚠️ Future option)
- **Requires**: Proper Lake/Lean setup
- **Commands**: `lake deps`, `lake graph`
- **Best for**: When Lean toolchain is fully configured

### Quick Start Recommendations:

1. **For immediate visualization**: Run the Python cells above
2. **For professional documentation**: Set up leanblueprint
3. **For publication graphics**: Use Graphviz with the generated DOT files

The analysis above should give you a complete picture of your project's dependency structure!